1DCNN -> 1DResNet  
loss: 0.0104 - avg_precision: 0.7206 - val_loss: 0.0104 - val_avg_precision: 0.7207 - lr: 5.0000e-05  
学習時間約8時間半  
Polars.DataFrame -> np.ndarrayの変換忘れで落ちかけるも、モデルの重みデータ改修して復旧  
LB 0.401

1dse+x epoch8 
loss: 0.0114 - avg_precision: 0.6824 - val_loss: 0.0136 - val_avg_precision: 0.6122 - lr: 0.0010  
LB:0.365 9時間MAX使って８Epochしか回せてないのでlossが落ち切ってなかったっぽい  
後半余裕ができ次第lr変えて続きやってもいいかも  
もしかしたらスタートのlrはもうちょい高くてもいいかな？

0: 293656924 samples
1: 1589906 samples

SeXc_fold0 : 8Epoch LB:0.410 arc:[8,8,2] 1h47m/epoch

SeXc_fold1_01: 2Epoch LB:0.376 archtecture:[4,2,2,2] 1Epoch 1h10m

In this notebook we will train a deep learning model using all the data available !
* preprocessing : I encoded the smiles of all the train & test set and saved it [here](https://www.kaggle.com/datasets/ahmedelfazouan/belka-enc-dataset) , this may take up to 1 hour on TPU.
* Training & Inference : I used a simple 1dcnn model trained on 20 epochs.

How to improve :
* Try a different architecture : I'm able to get an LB score of 0.604 with minor changes on this architecture.
* Try another model like Transformer, or LSTM.
* Train for more epochs.
* Add more features like a one hot encoding of bb2 or bb3.
* And of course ensembling with GBDT models.

In [1]:
!pip install fastparquet -q
!pip install polars
!pip install pyarrow


[notice] A new release of pip is available: 23.0.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 35.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 MB 11.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import gc
import os
import pickle
import random
import joblib
import numpy as np
import polars as pl
import pandas as pd
import pyarrow
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score as APS

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import (
    Conv1D, 
    BatchNormalization, 
    ReLU, 
    Add, 
    Input, 
    Embedding, 
    GlobalMaxPooling1D, 
    GlobalAveragePooling1D,
    Reshape,
    Multiply,
    Dense,
    Dropout
)


D0628 16:04:40.160850845      14 config.cc:119]                        gRPC EXPERIMENT tcp_frame_size_tuning               OFF (default:OFF)
D0628 16:04:40.160874108      14 config.cc:119]                        gRPC EXPERIMENT tcp_rcv_lowat                       OFF (default:OFF)
D0628 16:04:40.160877526      14 config.cc:119]                        gRPC EXPERIMENT peer_state_based_framing            OFF (default:OFF)
D0628 16:04:40.160880105      14 config.cc:119]                        gRPC EXPERIMENT flow_control_fixes                  ON  (default:ON)
D0628 16:04:40.160882438      14 config.cc:119]                        gRPC EXPERIMENT memory_pressure_controller          OFF (default:OFF)
D0628 16:04:40.160884992      14 config.cc:119]                        gRPC EXPERIMENT unconstrained_max_quota_buffer_size OFF (default:OFF)
D0628 16:04:40.160887348      14 config.cc:119]                        gRPC EXPERIMENT new_hpack_huffman_decoder           ON  (default:ON)
D0628 16:04:40.

In [3]:
class CFG:
    DEBUG = False
    PREPROCESS = False
    EPOCHS = 7
    BATCH_SIZE = 4096
    LR = 1e-4
    WD = 0.05
    
    N_ROWS = None
    
    NBR_FOLDS = 15
    SELECTED_FOLDS = [1]
    
    ON_TPU = False
    
    if DEBUG:
        EPOCHS = 3
        BATCH_SIZE = 1024
        NBR_FOLDS = 5
        N_ROWS = 100_000
    
    CHANNELS = 256
    EMB_SIZE = 128
    
    
    SEED = 2024
    
    BASE_DIR = '/kaggle/working/'
    LOG_DIR = os.path.join(BASE_DIR, 'logs')
    ADD_LEARN = True


In [4]:
%mkdir {CFG.LOG_DIR}

In [5]:
def set_seeds(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)

set_seeds(seed=CFG.SEED)

In [6]:
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect(tpu="local") # "local" for 1VM TPU
    strategy = tf.distribute.TPUStrategy(tpu)
    print("Running on TPU")
    CFG.ON_TPU = True
    print(f'ON_TPU : {CFG.ON_TPU}')    
    print("REPLICAS: ", strategy.num_replicas_in_sync)
except tf.errors.NotFoundError:
    print("Not on TPU")

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU

# Preprocessing

In [7]:
if CFG.PREPROCESS:
    enc = {'l': 1, 'y': 2, '@': 3, '3': 4, 'H': 5, 'S': 6, 'F': 7, 'C': 8, 'r': 9, 's': 10, '/': 11, 'c': 12, 'o': 13,
           '+': 14, 'I': 15, '5': 16, '(': 17, '2': 18, ')': 19, '9': 20, 'i': 21, '#': 22, '6': 23, '8': 24, '4': 25, '=': 26,
           '1': 27, 'O': 28, '[': 29, 'D': 30, 'B': 31, ']': 32, 'N': 33, '7': 34, 'n': 35, '-': 36}
    train_raw = pd.read_parquet('/kaggle/input/leash-BELKA/train.parquet')
    smiles = train_raw[train_raw['protein_name']=='BRD4']['molecule_smiles'].values
    assert (smiles!=train_raw[train_raw['protein_name']=='HSA']['molecule_smiles'].values).sum() == 0
    assert (smiles!=train_raw[train_raw['protein_name']=='sEH']['molecule_smiles'].values).sum() == 0
    def encode_smile(smile):
        tmp = [enc[i] for i in smile]
        tmp = tmp + [0]*(142-len(tmp))
        return np.array(tmp).astype(np.uint8)

    smiles_enc = joblib.Parallel(n_jobs=96)(joblib.delayed(encode_smile)(smile) for smile in tqdm(smiles))
    smiles_enc = np.stack(smiles_enc)
    train = pd.DataFrame(smiles_enc, columns = [f'enc{i}' for i in range(142)])
    train['bind1'] = train_raw[train_raw['protein_name']=='BRD4']['binds'].values
    train['bind2'] = train_raw[train_raw['protein_name']=='HSA']['binds'].values
    train['bind3'] = train_raw[train_raw['protein_name']=='sEH']['binds'].values
    train.to_parquet('train_enc.parquet')

    test_raw = pd.read_parquet('/kaggle/input/leash-BELKA/test.parquet')
    smiles = test_raw['molecule_smiles'].values

    smiles_enc = joblib.Parallel(n_jobs=96)(joblib.delayed(encode_smile)(smile) for smile in tqdm(smiles))
    smiles_enc = np.stack(smiles_enc)
    test = pd.DataFrame(smiles_enc, columns = [f'enc{i}' for i in range(142)])
    test.to_parquet('test_enc.parquet')

else:
    if CFG.DEBUG:
        train = pl.read_parquet('/kaggle/input/belka-enc-dataset/train_enc.parquet', n_rows=CFG.N_ROWS)
        test = pl.read_parquet('/kaggle/input/belka-enc-dataset/test_enc.parquet', n_rows=CFG.N_ROWS)
    else:
        train = pl.read_parquet('/kaggle/input/belka-enc-dataset/train_enc.parquet')
        test = pl.read_parquet('/kaggle/input/belka-enc-dataset/test_enc.parquet')

# Modeling

In [8]:
class ModelArgs:
    input_dim = CFG.EMB_SIZE
    channels: int = CFG.CHANNELS 
    conv_kernel_size: int = 3
    seq_length: int = 142
    num_layers: list = [4,2,2,2]
    dropout_rate: float = 0.2
    num_classes: int = 3
    vocab_size: int = 37
    activation: str = 'silu'
    se_reduction: int = 16 

In [9]:
class ConvSiluBN(layers.Layer):
    def __init__(self, out_size:int,kernel:int=3, strides:int=1, act:bool=True):
        super().__init__()
        self.act = act
        self.bn= layers.BatchNormalization()
        initializer = keras.initializers.HeNormal(CFG.SEED)
        self.conv = layers.SeparableConv1D(out_size, kernel, strides=strides, padding='same',depthwise_initializer=initializer, pointwise_initializer=initializer)
        
    def call(self, x):
        if self.act:
            x = keras.activations.relu(x)
        x = self.bn(x)
        x = self.conv(x)
        return x


In [10]:
class XceptionLayer(layers.Layer):
    def __init__(self,  out_size_1:int, out_size_2=None,strides:int=1, act:bool=True, pool:bool=False):
        super().__init__()
        if out_size_2 is None:
            out_size_2 = out_size_1
        
        self.conv1 = ConvSiluBN(out_size_1, act=act)
        self.conv2 = ConvSiluBN(out_size_2)
        if pool:
            self.fc = layers.MaxPool1D(pool_size=3, strides=strides, padding='same')
        else:
            self.fc = ConvSiluBN(out_size_2, strides=strides)
        self.se = SELayer(out_size_2)
    
    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.fc(x)
        x = self.se(x)
        return x
        
        
            

In [11]:
class SELayer(layers.Layer):
    def __init__(self,input_size:int , reduction:int=16):
        super().__init__()
        self.avg_pool = layers.GlobalAveragePooling1D(keepdims=True)
        self.dense1 = layers.Dense(
            input_size // reduction, 
            activation='silu',
            use_bias = False
        )
        self.dense2 = layers.Dense(
            input_size, 
            activation='sigmoid',
            use_bias = False
        )
    
    def call(self, x):
        y = self.avg_pool(x)
        y = self.dense1(y)
        y = self.dense2(y)
        
        return x * y

In [12]:
class ResidualLayer(layers.Layer):
    def __init__(self, input_size:int, out_size:int, strides:int=1, pool:bool=False):
        super().__init__()
        self.norm = layers.LayerNormalization(epsilon=1e-5)
        self.xcp = XceptionLayer(out_size,strides=strides,pool=pool)
        self.in_eq_out = input_size == out_size
        if not self.in_eq_out :
            self.conv = layers.Conv1D(out_size,1, strides=strides, padding='same', kernel_initializer='he_normal')
        self.add = layers.Add()
            
    
    def call(self, x):
        res = x
        if not self.in_eq_out:
            res = self.conv(res)
        
        
        x = self.norm(x)
        x = self.xcp(x)
        x = self.add([x, res])
        return x        

In [13]:
class ResidualBlock(layers.Layer):
    def __init__(self,in_size, out_size, num_layer, strides):
        super().__init__()
        self.layers = [ResidualLayer(in_size,in_size) for _ in range(num_layer-1)]
        self.last_layer = ResidualLayer(in_size, out_size, strides=strides, pool=True)
    
    def call(self,x):
        for layer in self.layers:
            x = layer(x)
        x = self.last_layer(x)
        return x

In [14]:
def make_tpu_model(args: ModelArgs):
    with strategy.scope():
        inputs = keras.Input(shape=(args.seq_length,), dtype='int32')
        x = layers.Embedding(input_dim=args.vocab_size, output_dim=args.input_dim, input_length=args.seq_length, mask_zero = True)(inputs)
        x = layers.Conv1D(args.channels, 1,padding='same',activation='relu', kernel_initializer='he_normal')(x)
        x = layers.BatchNormalization()(x)

        for i, layer_size in enumerate(args.num_layers, 1):
            strides = 2
            if i % 2 == 0:
                strides = 1
            x = ResidualBlock(args.channels*i,args.channels*(i+1), layer_size, strides)(x)
            x = layers.Dropout(0.1)(x)

        x = layers.LayerNormalization(epsilon=1e-5)(x)

        x = layers.GlobalMaxPooling1D()(x)

        x = layers.Dense(1024, activation='relu')(x)
        x = layers.Dropout(0.1)(x)
        x = layers.Dense(512, activation='relu')(x)
        x = layers.Dropout(0.1)(x)
        x = layers.Dense(512, activation='relu')(x)
        x = layers.Dropout(0.1)(x)

        outputs = Dense(3, activation='sigmoid')(x)

        model = tf.keras.models.Model(inputs = inputs, outputs = outputs)
        optimizer = tf.keras.optimizers.Adam(learning_rate=CFG.LR, weight_decay = CFG.WD)
        loss = 'binary_crossentropy'
        weighted_metrics = [tf.keras.metrics.AUC(curve='PR', name = 'avg_precision')]
        model.compile(
        loss=loss,
        optimizer=optimizer,
        weighted_metrics=weighted_metrics,
        )
        return model

model = make_tpu_model(ModelArgs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 142)]             0         
                                                                 
 embedding (Embedding)       (None, 142, 128)          4736      
                                                                 
 conv1d (Conv1D)             (None, 142, 256)          33024     
                                                                 
 batch_normalization (BatchN  (None, 142, 256)         1024      
 ormalization)                                                   
                                                                 
 residual_block (ResidualBlo  (None, 71, 512)          1198848   
 ck)                                                             
                                                                 
 dropout (Dropout)           (None, 71, 512)           0     

In [15]:
if CFG.ADD_LEARN:
    model.load_weights('/kaggle/input/xse_v3/tensorflow2/fold1_01/1/model-1_SeXc_01.h5')

# Train & Inference

In [16]:
FEATURES = [f'enc{i}' for i in range(142)]
TARGETS = ['bind1', 'bind2', 'bind3']
skf = StratifiedKFold(n_splits = CFG.NBR_FOLDS, shuffle = True, random_state = 42)
fold_indexes = []
all_preds = []
for fold,(train_idx, valid_idx) in enumerate(skf.split(np.arange(len(train)), train[TARGETS].sum_horizontal())):
    
    if fold not in CFG.SELECTED_FOLDS:
        fold_indexes.append(valid_idx)
        continue;
    
    X_train = train[train_idx, FEATURES].to_numpy()
    y_train = train[train_idx, TARGETS].to_numpy()
    X_val = train[valid_idx, FEATURES].to_numpy()
    y_val = train[valid_idx, TARGETS].to_numpy()

    tensorboard_callback = keras.callbacks.TensorBoard(log_dir=CFG.LOG_DIR)
    es = tf.keras.callbacks.EarlyStopping(patience=5, monitor="val_loss", mode='min', verbose=1)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(monitor='val_loss', filepath=f"model-{fold}.h5",
                                                        save_best_only=True, save_weights_only=True,
                                                    mode='min')
    reduce_lr_loss = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.05, patience=5, verbose=1)
#     model = my_model()
    history = model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=CFG.EPOCHS,
            callbacks=[checkpoint, reduce_lr_loss, es, tensorboard_callback],
            batch_size=CFG.BATCH_SIZE,
            verbose=1,
        )
    model.load_weights(f"model-{fold}.h5")
    oof = model.predict(X_val, batch_size = 2*CFG.BATCH_SIZE)
    print('fold :', fold, 'CV score =', APS(y_val, oof, average = 'micro'))
    
    preds = model.predict(test.to_numpy(), batch_size = 2*CFG.BATCH_SIZE)
    all_preds.append(preds)

print('train_finish')
preds = np.mean(all_preds, 0)

# kf = pd.DataFrame(fold_indexes)
# kf.to_csv('/kaggle/working/skf.csv')

Epoch 1/7


2024-06-28 16:07:24.632728: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add_396/ReadVariableOp.
2024-06-28 16:07:25.622975: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add_396/ReadVariableOp.


22426/22426 [==============================] - ETA: 0s - loss: 0.0116 - avg_precision: 0.6723

2024-06-28 17:19:03.692814: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.
2024-06-28 17:19:03.912206: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.


22426/22426 [==============================] - 4434s 194ms/step - loss: 0.0116 - avg_precision: 0.6723 - val_loss: 0.0117 - val_avg_precision: 0.6870 - lr: 1.0000e-04
Epoch 2/7
22426/22426 [==============================] - 4302s 192ms/step - loss: 0.0112 - avg_precision: 0.6886 - val_loss: 0.0110 - val_avg_precision: 0.6980 - lr: 1.0000e-04
Epoch 3/7
22426/22426 [==============================] - 4351s 194ms/step - loss: 0.0110 - avg_precision: 0.6983 - val_loss: 0.0110 - val_avg_precision: 0.7018 - lr: 1.0000e-04
Epoch 4/7
22426/22426 [==============================] - 4349s 194ms/step - loss: 0.0108 - avg_precision: 0.7053 - val_loss: 0.0107 - val_avg_precision: 0.7098 - lr: 1.0000e-04
Epoch 5/7
22426/22426 [==============================] - 4361s 194ms/step - loss: 0.0107 - avg_precision: 0.7106 - val_loss: 0.0107 - val_avg_precision: 0.7119 - lr: 1.0000e-04
Epoch 6/7
22426/22426 [==============================] - 4344s 194ms/step - loss: 0.0106 - avg_precision: 0.7146 - val_loss: 

2024-06-29 00:35:17.546516: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2024-06-29 00:35:17.728067: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


801/801 [==============================] - 101s 110ms/step
fold : 1 CV score = 0.7169305243464026
205/205 [==============================] - 29s 139ms/step
train_finish


# Submission

In [17]:
tst = pd.read_parquet('/kaggle/input/leash-BELKA/test.parquet')
tst['binds'] = 0
tst.loc[tst['protein_name']=='BRD4', 'binds'] = preds[(tst['protein_name']=='BRD4').values, 0]
tst.loc[tst['protein_name']=='HSA', 'binds'] = preds[(tst['protein_name']=='HSA').values, 1]
tst.loc[tst['protein_name']=='sEH', 'binds'] = preds[(tst['protein_name']=='sEH').values, 2]
tst[['id', 'binds']].to_csv('submission.csv', index = False)